In [1]:
# Importar librerías para tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd  
import re

# Importar librerías para web scraping y manipulación de datos
# -----------------------------------------------------------------------
from bs4 import BeautifulSoup
import requests

# Importar librerías para automatización de navegadores web con Selenium
# -----------------------------------------------------------------------
from selenium import webdriver  
from webdriver_manager.chrome import ChromeDriverManager  
from selenium.webdriver.common.keys import Keys  
from selenium.webdriver.support.ui import Select 

# Importar librerías para pausar la ejecución
# -----------------------------------------------------------------------
from time import sleep  


# Recuperamos la lista final de tuplas

In [2]:
# Traer resultado tuplas total extraídos en la Fase 1
df = pd.read_csv('archivo_final_actualizado.csv', header=None)
lista_tuplas = [tuple(row) for row in df.values]

In [3]:
lista_tuplas = lista_tuplas[1:]

In [4]:
lista_tuplas[:3]

[('6,9',
  '72',
  'Jon Favreau',
  'Justin Theroux',
  'Con el mundo sabiendo que es Iron Man, Tony Stark debe lidiar tanto con su salud en declive como con un loco vengativo con vínculos con el legado de su padre.',
  "['Acción', 'Ciencia ficción']",
  '2h 4min',
  'Iron Man 2',
  'tt1228705'),
 ('6,0',
  '21',
  'Florian Henckel von Donnersmarck',
  'Florian Henckel von Donnersmarck',
  'Frank es un turista estadounidense que visita Italia para reparar un corazón roto. Elise es una mujer extraordinaria que se cruza deliberadamente en su camino.',
  "['Acción', 'Suspense']",
  '1h 43min',
  'The Tourist',
  'tt1243957'),
 ('7,0',
  '72',
  'Robert Schwentke',
  'Jon Hoeber',
  'Cuando su pacífica vida se ve amenazada por un asesino de alta tecnología, el exagente de operaciones encubiertas Frank Moses vuelve a reunir a su antiguo equipo en un último esfuerzo por sobrevivir y descubrir a sus agresores.',
  "['Acción', 'Comedia', 'Crimen']",
  '1h 51min',
  'RED',
  'tt1245526')]

In [5]:
len(lista_tuplas)

377

In [6]:
count = 0
for i in lista_tuplas:
    if type(i[-1]) == float:
        count += 1

print(count)

101


In [7]:
for i in lista_tuplas:
    if type(i[-1]) == float:
        lista_tuplas.remove(i)

print(len(lista_tuplas))

305


In [8]:
len(lista_tuplas)

305

In [9]:
# Bucle for para realizar pruebas
for peli in lista_tuplas[:]: # Prueba: Iterando para extraer el id_imdb y el nombre 
    id_imdb = peli[-1]
    nombre = peli[-2]
    print(id_imdb)
    print(nombre)
    break

tt1228705
Iron Man 2


In [10]:
# Creación de las variables que vamos a usar en la extración con Selenium
url_imdb = "https://www.imdb.com/"
selector_cookies_imdb = "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd"

In [11]:
# Creacción de las listas donde vamos a almacenar los resultados de la extración.
resultados_completos = []
resultados_parciales = [] # Nos permitimos añadir tuplas con hasta 2 valores nulos.
tuplas_tabla_intermedia = []

In [12]:
# Creamos una función para abrir el navegador 
def abrir_navegador(driver, url, selector):
    driver = webdriver.Chrome() # Inicializa el navegador Chrome
    sleep(3)
    driver.maximize_window() # Maximiza la ventana
    driver.get(url) # Navega a la url pasada por parámetro
    sleep(3)
    try:
        driver.find_element("css selector", selector).click() # Intentamos aceptar las cookies
        sleep(3)
    except: # Si salta error, imprime un mensaje avisando.
        print("No ha sido posible aceptar las cookies, puede que no saltó el aviso o que se haya cambiado el selector.")
    
    return driver # retorna el driver que será usado para las demás tareas

In [14]:
# Creación de algunas variables que vamos a usar en la extración
url_imdb = "https://www.imdb.com/"
selector_cookies_imdb = "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd"

# Listas de selectores CSS para hacer iteraciones
selector_nombre_que_hace = ["#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > section > div:nth-child(5) > section > section > div.sc-491663c0-3.bdjVSf > div"]
selector_anio_nasc = ["#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > section > div:nth-child(5) > section > section > div.sc-491663c0-4.yjUiO > div.sc-491663c0-6.lnlBxO > div.sc-491663c0-11.cvvyMK > section > aside > div > span:nth-child(2)"]
selector_conocido_por = ["#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > div.celwidget > section:nth-child(1) > div.sc-1c5b3162-0.fFgRFS > div > div.ipc-sub-grid.ipc-sub-grid--page-span-2.ipc-sub-grid--wraps-at-above-l.ipc-shoveler__grid"]

selector_awards_click = ["#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > section > div:nth-child(5) > section > section > div.sc-66ec1b32-0.kbbKze > div > div:nth-child(2) > ul > li:nth-child(2) > a"]
# Hay que hacer .click() en el selector awards y luego driver.back()

selector_awards = "#jump-to"

# Selectores de actores
select_actor1 = ["#fullcredits_content > table.cast_list > tbody > tr:nth-child(2) > td:nth-child(2) > a"]
select_actor2 = ["#fullcredits_content > table.cast_list > tbody > tr:nth-child(3) > td:nth-child(2) > a"]
select_actor3 = ["#fullcredits_content > table.cast_list > tbody > tr:nth-child(4) > td:nth-child(2) > a"]
select_actor4 = ["#fullcredits_content > table.cast_list > tbody > tr:nth-child(5) > td:nth-child(2) > a"]
select_actor5 = ["#fullcredits_content > table.cast_list > tbody > tr:nth-child(6) > td:nth-child(2) > a"]
select_actor6 = ["#fullcredits_content > table.cast_list > tbody > tr:nth-child(7) > td:nth-child(2) > a"]
select_actor7 = ["#fullcredits_content > table.cast_list > tbody > tr:nth-child(8) > td:nth-child(2) > a"]
select_actor8 = ["#fullcredits_content > table.cast_list > tbody > tr:nth-child(9) > td:nth-child(2) > a"]
select_actor9 = ["#fullcredits_content > table.cast_list > tbody > tr:nth-child(10) > td:nth-child(2) > a"]
select_actor10 = ["#fullcredits_content > table.cast_list > tbody > tr:nth-child(11) > td:nth-child(2) > a"]
lista_select_actores = [select_actor1, select_actor2, select_actor3, select_actor4, select_actor5, select_actor6, select_actor7, select_actor8, select_actor9, select_actor10]

# Selectores de premios
select_premios = ["#__next > main > div > section > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child(3)", "#__next > main > div > section > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child(4)", "#__next > main > div > section > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child(5)", "#__next > main > div > section > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child(6)", "#__next > main > div > section > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child(7)"]


In [15]:
# Creamos una función para iterar por los selectores haciendo los try except para extraer datos de imdb
def extrae_datos(lista_selector, driver):
    for selector in lista_selector:
        try:
            dato = driver.find_element("css selector", selector).text
            sleep(2)
            return dato 
        except: #Exception as e:
            # excepcion = e
            pass

    # error = f"No se ha podido extraer el dato, error de tipo: {excepcion}."
    # not_found.append(error)
    dato = None
    return dato

In [16]:
# Creamos una función para clicar en elementos
def clicar(lista_selector, driver):
    for selector in lista_selector:
        try:
            driver.find_element("css selector", selector).click()
            return True
        except:
            print(f"No ha sido posible clicar.")
            return False

In [17]:
# Creamos una función para extraer los premios y luego limpiar los datos si tenemos 5 o más premios
def extrae_premios(lista_selector, driver):
    lista_premios = []
    for selector in lista_selector:
        try:
            dato = driver.find_element("css selector", selector).text
            sleep(2)
            lista_premios.append(dato)
        except:
            pass
    
    try:
        if len(lista_premios) > 0:
            lista_premios_limpios = []
            for premio in lista_premios:
                premio_limpio = premio.split('\n')
                premio_limpio = premio_limpio[0].replace(',', '') + ' ' + premio_limpio[1][5:8]
                lista_premios_limpios.append(premio_limpio)
        else:
            return None
    except:
        return None

    if 'Contribute to this page st ' in lista_premios_limpios:
        lista_premios_limpios.remove('Contribute to this page st ')
    if 'More from this person  wo' in lista_premios_limpios:
        lista_premios_limpios.remove('More from this person  wo')
        
    if len(lista_premios_limpios) >= 5:
        return lista_premios_limpios[:5]
    else:
        return lista_premios_limpios

In [17]:
# Llamamos a todas las funciones para extraer los datos

driver1 = None
driver1 = abrir_navegador(driver1, url_imdb, selector_cookies_imdb)

contador = 0 # Creamos un contador para saber por que iteración vamos

# Iterar por los ids_imdbs para pasarlos por el campo search de la web de imdb:
for peli in lista_tuplas[:100]:
    id_imdb = peli[-1]
    contador += 1
    print(f"Vamos por la iteración: {contador}.")

    # Pasamos el id_imdb al buscador de imdb
    driver1.find_element("css selector", "#suggestion-search").send_keys(id_imdb, Keys.ENTER)
    sleep(5)

    # Pinchamos en cast & crew            
    driver1.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > section > div:nth-child(5) > section > section > div.sc-66ec1b32-0.kbbKze > div > div:nth-child(2) > ul > li:nth-child(1) > a").click()
    sleep(3)


    for select_actor in lista_select_actores:    
        # Creamos todas las variables que vamos a guardar en la tupla
        nombre = que_hace = anio_nasc = conocido_por = premios = None

        # Pinchamos en el actor
        pinchar_actor = clicar(select_actor, driver1)
        sleep(4)
        if pinchar_actor:
            pass
        else:
            continue
        
        # Extraímos los datos de imdb
        nombre_que_hace = extrae_datos(selector_nombre_que_hace, driver1)
        if nombre_que_hace:
            try:
                nombre = nombre_que_hace.split('\n')[0]
                nombre = re.sub(r'\(.*?\)', '', nombre)
            except:
                pass

            try:
                que_hace = nombre_que_hace.split('\n')[1:] # que_hace = extrae_datos(selector_nombre_que_hace, driver1)
            except:
                pass

        try:
            anio_nasc = extrae_datos(selector_anio_nasc, driver1)[-4:]
            patron = r'^\d{4}$'
            if re.match(patron, anio_nasc):
                pass
            else:
                anio_nasc = None
        except:
            pass
        
        try:
            conocido_por = extrae_datos(selector_conocido_por, driver1).split('\n')[::4]
        except:
            pass
        
        # Pinchamos en los awards
        pinchar_awards = clicar(selector_awards_click, driver1)
        sleep(6)

        # Extraemos los premios usando la función que extrae los premios
        if pinchar_awards:
            premios = extrae_premios(select_premios, driver1)
            sleep(2)
        else:
            driver1.back()
            sleep(4)
            continue
        
        # Creamos la tupla con todos los datos y la tupla para la tabla intermedia
        tupla = nombre, anio_nasc, conocido_por, que_hace, premios
        tupla_intermedia = id_imdb, nombre
        print("TUPLA:", tupla)
        print("TUPLA INTERMEDIA:", tupla_intermedia)
                    
        none_count = tupla.count(None) # Cuenta cuántos Nones hay en la tupla 
        if none_count > 2: # Si hay mas de 2 Nones, no hagas nada
            pass
        elif none_count > 0:
            if None in tupla_intermedia: # Si hay un None en la tupla intermedia
                pass
            else:
                tuplas_tabla_intermedia.append(tupla_intermedia)
                resultados_parciales.append(tupla)
        else:
            if None in tupla_intermedia:
                pass
            else:
                tuplas_tabla_intermedia.append(tupla_intermedia)
                resultados_completos.append(tupla)
        
        driver1.back()
        sleep(4)
        driver1.back()
        sleep(5)

driver1.close()

Vamos por la iteración: 1.
TUPLA: ('Robert Downey Jr.', '1965', ['Iron Man', 'Iron Man 3', 'Sherlock Holmes', 'Vengadores: Endgame'], ['Reparto', 'Producción', 'Guion'], ['Academy Awards USA Gan', 'Academy of Science Fiction Fantasy & Horror Films USA Nom', 'Boston Society of Film Critics Awards Nom', 'BAFTA Awards Gan', 'Critics Choice Awards Gan'])
TUPLA INTERMEDIA: ('tt1228705', 'Robert Downey Jr.')
TUPLA: ('Gwyneth Paltrow', '1972', ['Shakespeare in Love (Shakespeare enamorado)', 'Dos vidas en un instante', 'Emma', 'Iron Man 3'], ['Reparto', 'Producción', 'Dirección'], ['Academy Awards USA Gan', 'Academy of Science Fiction Fantasy & Horror Films USA Nom', 'Bambi Awards Gan', 'Blockbuster Entertainment Awards Gan', 'BAFTA Awards Nom'])
TUPLA INTERMEDIA: ('tt1228705', 'Gwyneth Paltrow')
TUPLA: ('Don Cheadle', '1964', ['Crash (Colisión)', 'El irlandés', 'Hotel Rwanda', 'Miles Ahead'], ['Reparto', 'Producción', 'Dirección'], ['Academy Awards USA Nom', 'Berlin International Film Festiva

In [ ]:
# Guardar los datos de manera permanente

df_resultados_completos = pd.DataFrame(resultados_completos)
df_resultados_parciales = pd.DataFrame(resultados_parciales)
df_tuplas_tabla_intermedia = pd.DataFrame(tuplas_tabla_intermedia)

df_resultados_completos.to_csv('resultados_completos_F3_laura.csv') # Cambiar nombre !!! 
df_resultados_parciales.to_csv('resultados_parciales_F3_laura.csv')
df_tuplas_tabla_intermedia.to_csv('tuplas_tabla_intermedia_F3_laura.csv') 

df_resultados_completos.to_json('resultados_completos_F3_laura.json') 
df_resultados_parciales.to_json('resultados_parciales_F3_laura.json')
df_tuplas_tabla_intermedia.to_json('tuplas_tabla_intermedia_F3_laura.json')
